In [1]:
from  selenium import webdriver 
from bs4 import BeautifulSoup as bs
import re 
import string
from time import  sleep
from selenium.common.exceptions import TimeoutException
import pickle
from collections import deque
from selenium.webdriver.common.by import By
import pandas as pd
import numpy
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
with open("./vilas_state.pkl",'rb') as file:
        vilas,stack2,link2=pickle.load(file)
df_vilas=pd.DataFrame(vilas)
df_vilas.rename(columns={'data': 'date'},inplace=True, errors='raise')
len(vilas)

21737

***Cleaning***


In [3]:
df_vilas.drop(['vlaues'],axis=1,inplace=True)
df_vilas.drop(df_vilas.loc[[type(v)!=str for v in df_vilas['date']]].index,inplace=True)
df_vilas.head()

,price,location,date,title,link,viwers,negotiable,Bedrooms,Bathrooms,Area (m²),Type,Payment Option,Compound,Delivery Term,Amenities,Furnished,Delivery Date,Down Payment
0,"7,400,000 EGP",Sheikh Zayed، Giza,9 March 2022,استغل الفرصه وامتلك اخر فيلا في ليك ويست بتسهي...,https://www.olx.com.eg/en/ad/8-IDcHny4.html,54,0.0,[4],[4],[338],"[Stand, Alone, Villa]","[Cash, or, Installment]","[Lake, West]","[Core, &, Shell]",NaN,NaN,NaN,NaN
1,"7,700,000 EGP",Rehab City، Cairo,1 March 2022,فيلا للبيع نموذج c بمدينة الرحاب تشطيب كامل,https://www.olx.com.eg/en/ad/c-IDcGhF0.html,291,0.0,[3],[3],[456],"[Stand, Alone, Villa]",[Cash],NaN,[Finished],"[Balcony, Pets, Allowed, Private, Garden, Secu...",[No],"[Ready, to, move]",NaN
2,"4,900,000 EGP",Shorouk City، Cairo,10 January 2022,تاون هاوس245م للبيع بأفضل نظم سداد استلام خلال...,https://www.olx.com.eg/en/ad/245-gcv-IDc4U6j.html,205,0.0,[4],[2],[245],"[Town, House]","[Cash, or, Installment]",[GCR],NaN,"[Balcony, Built, in, Kitchen, Appliances, Maid...",[No],[2022],NaN
3,"6,500,000 EGP","New Cairo - El Tagamoa, Cairo",10 March 2022,توين هاوس 255م للبيع - هايد بارك - التجمع,https://www.olx.com.eg/en/ad/255-IDcqiQS.html,67,0.0,[4],[4],[255],"[Twin, House]",[Cash],"[Hyde, Park, New, Cairo]","[Semi, Finished]","[Private, Garden, Security]",[No],"[Ready, to, move]",NaN
4,"12,000,000 EGP",Sheikh Zayed، Giza,27 February 2022,للبيع فيلا بسعر مميز علي الجولف كمبوند الربوة,https://www.olx.com.eg/en/ad/-IDcG0Qo.html,165,0.0,[4],[3],[800],"[Stand, Alone, Villa]",[Cash],"[Al, Rabwa]",[Finished],"[Balcony, Built, in, Kitchen, Appliances, Cove...",[No],"[Ready, to, move]",[1]


In [4]:
df_vilas['Down Payment'][10]

['312,500']

In [5]:
z=list(df_vilas.columns)
x=['price','location','date','title','link','viwers','negotiable','Amenities']
selected=[i for i in z if i not in x ]
selected

['Bedrooms',
 'Bathrooms',
 'Area (m²)',
 'Type',
 'Payment Option',
 'Compound',
 'Delivery Term',
 'Furnished',
 'Delivery Date',
 'Down Payment']

In [6]:
for i in selected:
    print(i)
    print(df_vilas[i].value_counts())
    print("*"*80)

Bedrooms
[4]      8826
[3]      4968
[5]      4224
[6]      1798
[7]       618
[10+]     536
[8]       235
[2]       168
[10]      148
[9]       135
[1]        47
Name: Bedrooms, dtype: int64
********************************************************************************
Bathrooms
[4]      7550
[3]      6340
[5]      3637
[6]      1537
[2]      1009
[7]       632
[10+]     410
[8]       240
[1]       153
[9]        98
[10]       97
Name: Bathrooms, dtype: int64
********************************************************************************
Area (m²)
[300]       548
[400]       504
[600]       484
[500]       469
[260]       401
           ... 
[1,375]       1
[443]         1
[609]         1
[1,301]       1
[388.78]      1
Name: Area (m²), Length: 1078, dtype: int64
********************************************************************************
Type
[Stand, Alone, Villa]    11528
[Town, House]             5770
[Twin, House]             4405
Name: Type, dtype: int64
******************

In [7]:
def x (z):
    try:
        return ' '.join(str(e) for e in z)
    except Exception as e:
        if type(z)==z:return 0
        print(z)
        return z 
for i in selected:
    df_vilas[i]=df_vilas[i].apply(x)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


In [8]:
type(df_vilas['Down Payment'][0])==numpy.float64

False

In [9]:
df_vilas['Amenities'][0]==None

False

In [10]:
for i in selected:
    print(i)
    print(df_vilas[i].value_counts())
    print("*"*80)

Bedrooms
4      8826
3      4968
5      4224
6      1798
7       618
10+     536
8       235
2       168
10      148
9       135
1        47
Name: Bedrooms, dtype: int64
********************************************************************************
Bathrooms
4      7550
3      6340
5      3637
6      1537
2      1009
7       632
10+     410
8       240
1       153
9        98
10       97
Name: Bathrooms, dtype: int64
********************************************************************************
Area (m²)
300       548
400       504
600       484
500       469
260       401
         ... 
1,375       1
443         1
609         1
1,301       1
388.78      1
Name: Area (m²), Length: 1078, dtype: int64
********************************************************************************
Type
Stand Alone Villa    11528
Town House            5770
Twin House            4405
Name: Type, dtype: int64
********************************************************************************
Payment Option


In [11]:
def x(z):
    if z.endswith('+'):
        return int(z[:-1])
    else:
        return int(z)
df_vilas['Bathrooms']=df_vilas['Bathrooms'].apply(x)
df_vilas['Bedrooms']=df_vilas['Bedrooms'].apply(x)

In [12]:
def x(z):
    if type(z)==float:
        return None
    else:
        x=str()
    for i in z:
        x+=i+','
    return x[:-1]  
        
df_vilas['Amenities']=df_vilas['Amenities'].apply(x)

In [13]:
def x(z):
    try:
        x=''
        if type(z)==str: 
            for i in z:
                if  i.isdigit():
                    x+=i
            return float(x)
        elif type(z)==list:
            print(z,'list')
            for i in z[0]:
                if i.isdigit():
                    x+=i
            return float(x)
        else:
            return 0.0
    except:
        print(z)
        return 0.0
df_vilas['price']=df_vilas['price'].apply(x)
df_vilas['Down Payment']=df_vilas['Down Payment'].apply(x)
df_vilas['Area (m²)']=df_vilas['Area (m²)'].apply(x)
df_vilas[['price','Down Payment','Area (m²)']]

,price,Down Payment,Area (m²)
0,7400000.0,0.0,338.0
1,7700000.0,0.0,456.0
2,4900000.0,0.0,245.0
3,6500000.0,0.0,255.0
4,12000000.0,1.0,800.0
...,...,...,...
21732,15380000.0,0.0,630.0
21733,750000.0,750000.0,250.0
21734,5500000.0,0.0,250.0
21735,5500000.0,0.0,250.0


In [14]:
for i in list(df_vilas.columns):
    print(df_vilas[i].value_counts())

6500000.0     473
10000000.0    442
9000000.0     424
8000000.0     424
7500000.0     410
             ... 
2200179.0       1
8700002.0       1
10220002.0      1
8500003.0       1
6530000.0       1
Name: price, Length: 2408, dtype: int64
New Cairo - El Tagamoa، Cairo    6222
Sheikh Zayed، Giza               3104
New Cairo - El Tagamoa, Cairo    2550
6th of October، Giza             1987
Madinaty، Cairo                  1286
                                 ... 
Ras Sedr, South Sinai               1
New Sohag, Sohag                    1
Atssa، Fayoum                       1
Roushdy, Alexandria                 1
Haram, Giza                         1
Name: location, Length: 250, dtype: int64
 10 March 2022       695
 17 February 2022    674
 8 March 2022        627
 3 March 2022        619
 7 March 2022        606
 9 March 2022        589
 10 February 2022    579
 14 February 2022    578
 20 February 2022    578
 28 February 2022    568
 23 February 2022    565
 21 February 2022    550
 2

In [15]:
# df_vilas.info()
df_vilas['date']=df_vilas['date'].apply(lambda x: x.strip())
df_vilas['date']=pd.to_datetime(df_vilas['date'], format="%d %B %Y")

In [16]:
df_vilas['Area (m²)'] = df_vilas['Area (m²)'].astype(int)


In [17]:
df_vilas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21703 entries, 0 to 21736
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   price           21703 non-null  float64       
 1   location        21703 non-null  object        
 2   date            21703 non-null  datetime64[ns]
 3   title           21703 non-null  object        
 4   link            21703 non-null  object        
 5   viwers          21703 non-null  object        
 6   negotiable      21703 non-null  float64       
 7   Bedrooms        21703 non-null  int64         
 8   Bathrooms       21703 non-null  int64         
 9   Area (m²)       21703 non-null  int32         
 10  Type            21703 non-null  object        
 11  Payment Option  17664 non-null  object        
 12  Compound        13373 non-null  object        
 13  Delivery Term   11912 non-null  object        
 14  Amenities       15815 non-null  object        
 15  Fu

***Fill_Nulls***

In [18]:
for i ,v in df_vilas[df_vilas['Compound']=='Madinaty']['Amenities'].value_counts().iloc[[2]].items():
    items=i
items

'Balcony,Built,in,Kitchen,Appliances,Central,A/C,&,heating,Covered,Parking,Maids,Room,Pets,Allowed,Private,Garden,Security,Pool,Electricity,Meter,Water,Meter,Natural,Gas,Landline,Elevator'

In [19]:
# madanity=''
# for i, v in df_vilas.loc[(df_vilas['Compound']=='Madinaty') & (df_vilas['location']=='Madinaty, Cairo') & (df_vilas['Amenities']!='x')]['Amenities'].value_counts().items():
#     madanity=i
#     break
# madanity

In [20]:
# df_vilas.loc[(df_vilas['Amenities']=='x') & (df_vilas['location']=='Madinaty, Cairo'),'Compound']='Madinaty'
# df_vilas.loc[(df_vilas['Amenities']=='x') & (df_vilas['location']=='Madinaty, Cairo'),'Amenities']=madanity

***Feature Extraction***


In [21]:
# st=set()
# [[st.add(z) for  z in i ]for i in [x for x in  df_vilas['Amenities'] if type(x)!=float]]
# x=list(st)
# sub=['A/C','in','Meter','Private','Central','&','Built','Natural','Allowed', 'Room',]
# Amenities={i:0 for  i in [i for i in x if i not in sub]}
# print(Amenities.keys())

# def x(z):
#     Amenities2={'Pets': 0, 'Maids': 0, 'heating': 0, 'Balcony': 0, 'Garden': 0, 'Landline': 0, 'Security': 0, 'Parking': 0, 'Water': 0, 'Appliances': 0, 'Pool': 0, 'Gas': 0, 'Electricity': 0, 'Covered': 0, 'Kitchen': 0, 'Elevator': 0}
#     try:
#         if type(z)!=float:
#             for i in z:
#                 if i in Amenities2.keys():
#                     Amenities2[i]=1
#             return  pd.Series(list(Amenities2.values()))
#         return pd.Series(list(Amenities2.values()))
#     except Exception as e:
#         print(e)
#         print(z)
#         return pd.Series(list(Amenities2.values()))
# df_vilas[list(Amenities.keys())]=df_vilas['Amenities'].apply(x)

In [22]:
with open("./vlias-for-rent.pkl",'rb') as file:
        vilas_rent,stack2,link2=pickle.load(file)
df_vilas_rent=pd.DataFrame(vilas_rent)
df_vilas_rent.rename(columns={'data': 'date'},inplace=True, errors='raise')
df_vilas_rent.drop([i  for i in list(df_vilas_rent.columns) if i not in list(df_vilas.columns)],inplace=True,axis=1)
len(vilas_rent)

5142

In [23]:
df_vilas_rent.isnull().sum()

price                0
location             0
date                 0
title                0
link                 0
viwers               0
negotiable           0
Amenities         1255
Bedrooms            37
Bathrooms           37
Area (m²)           35
Furnished         1061
Type                15
Compound          2202
Down Payment      4335
Payment Option    5136
Delivery Date     5141
Delivery Term     5141
dtype: int64

In [24]:
with open("./apartmant.pkl",'rb') as file:
        apartmant,stack2,link2=pickle.load(file)
df_apartmant=pd.DataFrame(apartmant)
len(apartmant)

20317

In [25]:
df_apartmant

,price,location,data,title,link,viwers,negotiable,Bedrooms,Bathrooms,Area (m²),Level,Type,Payment Option,Amenities,Furnished,Delivery Date,Compound,Delivery Term,Down Payment
0,"2,560,000 EGP",Smoha، Alexandria,1 January 2022,شقة للبيع فى حى مروج 160م بمقدم 10% و اقساط عل...,https://www.olx.com.eg/en/ad/160-10-5-IDbV1PO....,67,0,[3],[2],[160],[7],[Apartment],[Installment],NaN,NaN,NaN,NaN,NaN,NaN
1,"1,650,000 EGP",Nasr City، Cairo,10 March 2022,للبيع شقه روعه ١٦٠م مدينه نصر,https://www.olx.com.eg/en/ad/-IDcHAPd.html,629,0,[3],[2],[160],[1],[Apartment],[Cash],"[Balcony, Built, in, Kitchen, Appliances, Priv...",[No],"[Ready, to, move]",NaN,NaN,NaN
2,"2,380,000 EGP","New Capital City, Cairo",13 February 2022,امتلك افضل شقة بالعاصمة تشطيب كامل بالمطبخ,https://www.olx.com.eg/en/ad/-IDcDVHS.html,28,1,[2],[2],[119],[Ground],[Apartment],"[Cash, or, Installment]","[Balcony, Covered, Parking, Maids, Room, Pets,...",NaN,"[Ready, to, move]",[Atika],[Finished],NaN
3,"665,000 EGP",New Cairo - El Tagamoa، Cairo,7 March 2022,شقه 130م بجاردن بكمبوند البروج فالشروق,https://www.olx.com.eg/en/ad/130-IDcH5kM.html,360,0,[2],[2],[130],[3],[Apartment],"[Cash, or, Installment]","[Balcony, Private, Garden, Security, Covered, ...",NaN,NaN,NaN,NaN,NaN
4,"3,900,000 EGP",Madinaty، Cairo,10 March 2022,بسعر لقطه ب مدينتى شقه للبيع نموذج 700 بافضل م...,https://www.olx.com.eg/en/ad/700-320-IDcHA2D.html,238,0,[4],[4],[320],[5],[Apartment],[Cash],NaN,[No],"[Ready, to, move]",[Madinaty],[Finished],NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20312,"1,750,000 EGP",New Cairo - El Tagamoa، Cairo,5 March 2022,شقة ١٩٠ م ب درة القاهرة أمام كمبوند اكاسيا أما...,https://www.olx.com.eg/en/ad/-IDcgTes.html,292,1,[3],[3],[190],[1],[Apartment],NaN,NaN,[No],NaN,NaN,NaN,NaN
20313,"900,000 EGP",Zarqa، Damietta,5 March 2022,شقة للبيع,https://www.olx.com.eg/en/ad/-IDcCi0O.html,77,1,[3],[2],[165],[1],[Apartment],[Installment],[Elevator],[No],NaN,NaN,NaN,"[30,000]"
20314,"1,850,000 EGP",Heliopolis، Cairo,5 March 2022,لراغبي الاستثمار بالتقسيط ارضي مدخل خاص 200م ن...,https://www.olx.com.eg/en/ad/200-IDcGO9M.html,237,0,[4],[2],[200],[Ground],[Apartment],"[Cash, or, Installment]","[Security, Electricity, Meter, Water, Meter]",[No],"[Ready, to, move]",NaN,"[Semi, Finished]",NaN
20315,"1,980,000 EGP",New Cairo - El Tagamoa، Cairo,5 March 2022,شقه للبيع في كمبوند جالريا مون فالي,https://www.olx.com.eg/en/ad/-IDcGO8w.html,111,0,[2],[2],[125],[2],[Apartment],"[Cash, or, Installment]","[Balcony, Covered, Parking, Pets, Allowed, Sec...",[No],"[Ready, to, move]","[Galleria, Moon, Valley]","[Semi, Finished]","[316,000]"


In [26]:
with open("./apertment-rent.pkl",'rb') as file:
        apertment_rent,stack2,link2=pickle.load(file)
df_apertment_rent=pd.DataFrame(apertment_rent)
len(apertment_rent)

21735

In [27]:
df_apertment_rent.isnull().sum()

price               0
location            0
data                0
title               0
link                0
viwers              0
negotiable          0
Bedrooms            0
Bathrooms           0
Area (m²)           0
Furnished        4687
Level               0
Type                4
Compound        15374
Amenities        7148
Down Payment    16927
dtype: int64

In [28]:
with open("state21.pkl",'rb') as file:
        state21,stack2,link2=pickle.load(file)
df_state21=pd.DataFrame(state21)
len(state21)

6518

In [29]:
df_state21.isnull().sum()

link               0
date               1
title              1
viwers             0
location           1
Price              1
sign               1
Amenities       2227
Bedrooms           1
Bathrooms          1
Area (m²)          1
Furnished       1444
Level              1
Type               1
Compound        4841
Down Payment    4928
table           6518
dtype: int64